In [ ]:
# Import libraries
from  sklearn.ensemble import RandomForestClassifier
from gensim.models import doc2vec
from collections import namedtuple

import numpy as np
from sklearn.metrics import f1_score,recall_score,precision_score,classification_report,confusion_matrix,accuracy_score

import json
import os
import slimit
from   slimit.lexer    import Lexer
from   slimit.parser   import Parser

import numpy as np
import itertools

# Some of the tokens that are excluded. These are generally punctuation.
exclude = ['LPAREN', 'RPAREN',  'PERIOD', 'SEMI', 'RBRACE', 'LBRACE', 'COMMA', 'LBRACKET', 'RBRACKET', 'COLON',  
           'NUMBER', 'EQ', 'LT', 'NOT', 'NE', 'EQEQ', 'PLUS', 'MULT','DIV', 'VAR', 'PLUSPLUS', 'STREQ', 'STRNEQ', 
           'NULL', 'MINUSMINUS', 'PLUSEQUAL', 'MOD', 'THIS', 'MINUS', 'RSHIFT', 'BAND', 'AND', 'LSHIFT', 'BOR', 'STRING', 
           'OR',  'TRUE', 'FALSE', 'LINE_TERMINATOR', 'DIV',  'VOID', 'EXOR', 'LE', 'GT' ]test_docs  = []

train_docs = []
train_labels= []
test_labels = []

def LoadData(filename):

   # Load data
   f = open(filename, "r")

   txt = f.read()

   obj = json.loads(txt)

   f.close()

   return obj

def Process_Files(filename, train_size, test_size):

   obj = LoadData(filename)

   print 'Loaded data'

   analyzedDocument = namedtuple('AnalyzedDocument', 'words tags')

   train_count_1=0
   train_count_0=0
   test_count_1=0
   test_count_0=0

   count = 0

   for i in itertools.count():
     try:

        x = obj[i]
        flg = 0

        if x["flag-any"] == 1 :
                       flg = 1

        if flg == 1 and (train_count_1 >=  train_size/2 and test_count_1 >= test_size/2):
           continue

        if flg == 0 and (train_count_0 >=  train_size/2 and test_count_0 >= test_size/2):
           continue


        filename = "../scripts/" + x["sha"] + ".js"

        filename = filename.strip('\n')

        # Open the Javascript file
        k = open(filename, "r+")

        # read from the file
        strng = k.read()
        # close the file
        k.close()

        
        words =[]

        # Tokenize  the javascript files
        lexer = Lexer()
        lexer.input(strng)

        for token in lexer:
           if  token.type in exclude:
              continue

           if token.type != 'ID':
              words.append(token.type)
           elif (token.type == 'ID' and len(token.value) > 2):
              words.append(token.value.lower())

        tags = [str(count+1)]

        count += 1
        if flg == 1:
           if (test_count_1 < test_size/2):
              test_labels.append(flg)
              test_docs.append(words)
                          test_count_1 += 1
           else:
              train_labels.append(flg)
              train_docs.append(analyzedDocument(words, tags))
              train_count_1 += 1
        else:
           if (test_count_0 < test_size/2):
              test_labels.append(flg)
              test_docs.append(words)
              test_count_0 += 1
           else:
              train_labels.append(flg)
              train_docs.append(analyzedDocument(words, tags))
              train_count_0 += 1

        if count >= train_size+test_size:
           break

     except:
       continue

   print 'Processed  %d files (train+test)' %count
   # Train model
   model = doc2vec.Doc2Vec(train_docs,  size=500, max_vocab_size=10000, min_count=1, window = 6, workers = 1, iter=20)

   print 'Doc2Vec Trained'
   return model

def  train_classify_javascript(filename, train_size, test_size):
  model = Process_Files(filename, train_size, test_size)
  train_list =  []
  test_list=[]
  for i in range(train_size):
     train_list.append(model.docvecs[i])

  for i in range(test_size):
     test_list.append(model.infer_vector(test_docs[i]))

  train_arrays=np.array(train_list)
  test_arrays=np.array(test_list)

  # Get the vectors
  classifier = RandomForestClassifier(n_estimators=300)
     classifier.fit(train_arrays, train_labels)
  print 'Random Forest Fitted'
  predict = classifier.predict(test_arrays)
  print classification_report(test_labels, predict)
  print classifier.score(test_arrays, test_labels)


train_classify_javascript('../scripts/table_flag.json', 14000, 1000)









[root@spark4 Models]# tail Rand.out
Doc2Vec Trained
Random Forest Fitted
             precision    recall  f1-score   support

          0       0.78      0.87      0.82       500
          1       0.85      0.75      0.80       500

avg / total       0.81      0.81      0.81      1000

0.81
